In [11]:
import re
import csv
import requests
import urllib
import json
import html
import time
from bs4 import BeautifulSoup

report_url_base = "https://suasprod.noc-science.at/XLCubedWeb/WebForm/ShowReport.aspx?rep="

indicators = { "1.C.1": {'name': "1.C.1", 'desc': "Drittmittelerlöse",
                         'report_path': "009 wissensbilanzen universitäten/003 intellektuelles vermögen - strukturkapital/002 1-c-1 erlöse forschung und entwicklung-projekte - projekte eek.xml",
                         'dimensions': ["Universitätsausrichtung", "Fördergeber", "Sitz", "Disziplin", "WZW"]},
               "1.C.2": {'name': "1.C.2", 'desc': "Forschungsinfrastruktur",
                         'report_path': "009 wissensbilanzen universitäten/003 intellektuelles vermögen - strukturkapital/003 1-c-2 investitionen in infrastruktur im f und e bereich.xml",
                         'dimensions': ["Art der FI", "Disziplin", "WZW"]},
               "2.B.1": {'name': "2.B.1", 'desc': "Strukturierte Doktorate",
                         'report_path': "009 wissensbilanzen universitäten/005 kernprozesse - forschung und entwicklung entwicklung und erschließung der künste/002 2-b-1 doktoratsstudierende mit beschäftigungsverhältnis zur universität.xml",
                         'dimensions': ["Personalkategorie", "Staat", "Universitätsausrichtung"]},
               "3.B.1": {'name': "3.B.1", 'desc': "Publikationen",
                         'report_path': "009 wissensbilanzen universitäten/007 output und wirkungen der kernprozesse - forschung und entwicklung- und erschließung der künste/001 3-b-1 wissenschaftliche künstlerische veröffentlichungen.xml",
                         'dimensions': ["WZW", "Disziplin", "Publikationstyp", "Universitätsausrichtung"]},
               "3.B.2": {'name': "3.B.2", 'desc': "Vorträge",
                         'report_path': "009 wissensbilanzen universitäten/007 output und wirkungen der kernprozesse - forschung und entwicklung- und erschließung der künste/002 3-b-2 gehaltene vorträge präsentationen.xml",
                         'dimensions': ["WZW", "Disziplin", "Veranstaltungstypus", "Vortragstypus", "Universitätsausrichtung"]},
               "3.B.3": {'name': "3.B.3", 'desc': "Patente",
                         'report_path': "009 wissensbilanzen universitäten/007 output und wirkungen der kernprozesse - forschung und entwicklung- und erschließung der künste/003 3-b-3 anzahl der patentanmeldungen.xml",
                         'dimensions': ["Transferleistung"]}}
for indicator in indicators.values():
    indicator['report_url'] = report_url_base + urllib.parse.quote(indicator['report_path'])
    
def create_filename(name, groups):
    return name + '_' + '_'.join(groups) + '.csv'
    
class Unidata:
    workbook = "https://suasprod.noc-science.at/XLCubedWeb/WebServices/WorkbookService.svc/"
    university_class = 's22'
    value_class = 's39'
    aggregate_category = 'Gesamt'
    years = {"2018": 0, "2017": 2, "2016": 4}
    cookie = None
    headers = {
        'Accept': '*/*',
        'Connection': 'keep-alive',
        'Host': 'suasprod.noc-science.at',
        'Referer': "https://suasprod.noc-science.at/XLCubedWeb/WebForm/ShowReport.aspx?rep=009+wissensbilanzen+universit%C3%A4ten/003+intellektuelles+verm%C3%B6gen+-+strukturkapital/002+1-c-1+erl%C3%B6se+forschung+und+entwicklung-projekte+-+projekte+eek.xml&toolbar=true",
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0',
        'X-Requested-With': 'XMLHttpRequest',
        'Content-Type': 'application/json; charset=utf-8',
        'Cookie': None,   
    }
    
    def __init__(self):
        self.load_url = self.workbook + "LoadAndRender"
        self.update_url = self.workbook + "UpdateAndRender"
    
    def create_table(self, indicator_name):
        indicator = indicators[indicator_name]
        print("Connecting to uni:data")
        if self.cookie is None:
            report_url = indicator['report_url']
            response = requests.get(report_url)
            self.cookie = response.headers['Set-Cookie'][:response.headers['Set-Cookie'].find(';')]
            self.headers['Cookie'] = self.cookie
        
        print("Creating table")
        report_path = indicator['report_path']
        data = {"Path":report_path,
                "renderMode":0,"Dpi":96,
                "ParameterNames":["rep"],
                "ParameterValues":[report_path]}
        r = requests.post(self.load_url, headers=self.headers, data=json.dumps(data))
        answer = r.json()
        return Table(self, indicator, answer)
    
class Table:
    def __init__(self, unidata, indicator, answer):
        self.unidata = unidata
        self.indicator = indicator
        tabID_find = "this.TabID = \""
        tabID = answer['d'][2]
        tabID = tabID[tabID.find(tabID_find)+len(tabID_find):tabID.find(tabID_find) + len(tabID_find) + 36]
        self.tableID = tabID
        self.groups = self.__get_groups(BeautifulSoup(answer['d'][0]))

    def __get_groups(self, soup):
        groups = {}
        for dim_id, dim_name in enumerate(self.indicator['dimensions']):
            dimslice = soup.find('div',attrs={'class': 'dimslice', 'dimensionslicerid': dim_id})
            groups[dim_name] = {'uid': dimslice['guid'],
                                'categories': {}}
            for idx, option in enumerate(dimslice.find_all('option')):
                if idx == 0:
                    groups[dim_name]['default_category'] = option['value']
                option_name = option.text
                i = 2
                while option_name in groups[dim_name]['categories']:
                    option_name = option.text + ' (' + str(i) + ')'
                    i+=1
                groups[dim_name]['categories'][option_name] = option['value']
        return groups
    
    def __change_category (self, group, category):
        data = {"TabID": self.tableID,
                "ReportPath": self.indicator['report_path'],
                "ServerEvent":"DimensionSlicerChanged",
                "ReportPathWithParameters": "null",
                "Names":["_Guid","_Member","_chartprops","_mapprops"],
                "Values":[group['uid'], category,"",""]}
        retval = requests.post(self.unidata.update_url, headers=self.unidata.headers, data=json.dumps(data))  
        time.sleep(0.5)
        return retval
    
    def __reset_group (self, group): 
        if self.unidata.aggregate_category in group['categories']:
            return self.__change_category(group, group['categories'][self.unidata.aggregate_category])
        else:
            return self.__change_category(group, group['default_category'])
    
    def __get_data_soup (self, group, category):   
        r = self.__change_category(group, category)
        answer = r.json()
        soup = BeautifulSoup(answer['d'][0])
        return soup
            
    def __get_table_data(self, soup):
        universities = {}
        for entry in soup.find_all('td', class_=self.unidata.university_class):
            values = entry.find_next_siblings('td', class_=self.unidata.value_class)
            university = {}
            for year, year_column in self.unidata.years.items():
                try:
                    university[year] = int(re.sub('[^\-0-9]','', values[year_column].text))
                except:
                    university[year] = None
            universities[entry.text.strip()] = university      
        return universities    

    def __get_data (self, groups, level = 0):
        assert(len(groups) >= 1)
        group = self.groups[groups[0]]
        data = {}
        for cat_name, cat_id in group['categories'].items():
            print('\t'*level + cat_name)
            if len(groups) == 1:
                soup = self.__get_data_soup(group, cat_id)
                data[cat_name] = self.__get_table_data(soup)
            else:
                self.__change_category(group, cat_name)
                data[cat_name] = self.__get_data(groups[1:], level+1)    
        self.__reset_group(group)
        return data    
    
    def __save_rows_helper(data, higher_groups, groups, csvw):
        for group_name, group in data.items():
            if len(groups) == 0:
                uni_name = group_name
                for year_name, value in group.items():
                    if value is not None:
                        csvw.writerow(higher_groups + [uni_name, year_name, value])
            else:
                Table.__save_rows_helper(group, higher_groups+[group_name], groups[1:], csvw)

    def __save_table_helper (data, groups, csvw):
        csvw.writerow(groups + ['Universität', 'Jahr', 'Wert'])
        groups.reverse()
        return Table.__save_rows_helper(data, [], groups, csvw)
    
    def download_all(self, groups, filename=None):
        print("Downloading data")
        data = self.__get_data(groups)
        if filename is None:
            filename = create_filename(indicator['name'], groups)
        with open(filename, 'w') as csv_file:
            csvw = csv.writer(csv_file, delimiter=';', lineterminator='\n')
            Table.__save_table_helper(data, groups, csvw)
        return filename
        
        
    



In [12]:
import ipywidgets as widgets
import os
from IPython.display import display
from IPython.display import FileLink, FileLinks, Javascript


    
class Selector:
    indicator_selector = widgets.Dropdown(
        options = [('Auswählen...', -1)] +
                  [(indicator['name']+' - '+indicator['desc'], indicator['name']) for indicator in indicators.values()],
    )
    dimension_selector = widgets.SelectMultiple(
        options = [('Bitte zuerst Kennzahl auswählen', -1)]
    )
    execute_button = widgets.Button(
        description='Laden', disabled=True,
        tooltip = 'Bitte Indikator und Dimensionen auswählen',
        button_style='info'
    )
    filename_text = widgets.Text(placeholder='Dateiname')
        
    indicator = None
    dimensions = None
    
    def indicator_selected(self, ind_sel):
        self.indicator = indicators[ind_sel['new']]
        self.dimension_selector.options = [(dim_name, idx) for idx, dim_name in enumerate(self.indicator['dimensions'])]

    def dimension_selected(self, dim_sel):
        self.dimensions = list(dim_sel['new'])
        if len(self.dimensions) == 0:
            self.execute_button.disabled = True
            self.execute_button.tooltip = 'Bitte Indikator und Dimensionen auswählen'
        else:
            self.execute_button.disabled = False
            self.execute_button.tooltip = 'Daten herunterladen'
            self.filename_text.value = create_filename(self.indicator['name'], [self.indicator['dimensions'][i] for i in self.dimensions])
        
    def execute(self, button):
        self.execute_button.disabled = True
        self.save()
        display(Javascript("Jupyter.notebook.execute_cells([2])"))
    
    def __init__(self):
        self.indicator_selector.observe(self.indicator_selected, names='value')
        self.dimension_selector.observe(self.dimension_selected, names='index')
        self.execute_button.on_click(self.execute)
        self.output = widgets.Output()
        self.box = widgets.VBox(
            [ widgets.HBox([widgets.Label("Kennzahl: "),
                            self.indicator_selector]),
              widgets.VBox([widgets.Label('Zu exportierende Dimensionen:'),
                               self.dimension_selector]),
             
              self.filename_text,
              self.execute_button])
        display(self.box, self.output)
        
            
    def save(self):
        print("Bitte warten...")
        unidata = Unidata()
        kennzahl = unidata.create_table(self.indicator['name'])
        file = kennzahl.download_all([self.indicator['dimensions'][i] for i in self.dimensions], self.filename_text.value)
        link = FileLink(file)
        display(link, self.output)
        
output = widgets.Output()
display(widgets.HTML('<h3>Neuer Datenexport</h3>'), output)
selector = Selector()


HTML(value='<h3>Neuer Datenexport</h3>')

Output()

Output()

Bitte warten...
Connecting to uni:data
Creating table
Gesamt
	Gesamt
	Agrarbiotechnologie, Lebensmittelbiotechnologie
	Andere Agrarwissenschaften
	Andere Geisteswissenschaften
	Andere Humanmedizin, Gesundheitswissenschaften
	Andere Naturwissenschaften
	Andere Sozialwissenschaften
	Andere Technische Wissenschaften
	Architektur
	Bauwesen
	Bildende Kunst
	Biologie
	Bühnengestaltung
	Chemie
	Chemische Verfahrenstechnik
	Computermusik
	Design
	Elektrotechnik, Elektronik, Informationstechnik
	Erziehungswissenschaften
	Film und Fernsehen
	Geowissenschaften
	Geschichte, Archäologie
	Gesundheitswissenschaften
	Humangeographie, Regionale Geographie, Raumplanung
	Industrielle Biotechnologie
	Informatik
	Interpretation - instrumental
	Interpretation - vokal
	Jazz/Improvisation
	Klinische Medizin
	Komposition
	Konservierung und Restaurierung
	Kunstwissenschaften
	Land- und Forstwirtschaft, Fischerei
	Maschinenbau
	Mathematik
	Medien- und Kommunikationswissenschaften
	Mediengestaltung
	Medizinische 

	Veterinärmedizin
	Werkstofftechnik
	Wirtschaftswissenschaften
Gemeinden und Gemeindeverbände (ohne Wien)
	Gesamt
	Agrarbiotechnologie, Lebensmittelbiotechnologie
	Andere Agrarwissenschaften
	Andere Geisteswissenschaften
	Andere Humanmedizin, Gesundheitswissenschaften
	Andere Naturwissenschaften
	Andere Sozialwissenschaften
	Andere Technische Wissenschaften
	Architektur
	Bauwesen
	Bildende Kunst
	Biologie
	Bühnengestaltung
	Chemie
	Chemische Verfahrenstechnik
	Computermusik
	Design
	Elektrotechnik, Elektronik, Informationstechnik
	Erziehungswissenschaften
	Film und Fernsehen
	Geowissenschaften
	Geschichte, Archäologie
	Gesundheitswissenschaften
	Humangeographie, Regionale Geographie, Raumplanung
	Industrielle Biotechnologie
	Informatik
	Interpretation - instrumental
	Interpretation - vokal
	Jazz/Improvisation
	Klinische Medizin
	Komposition
	Konservierung und Restaurierung
	Kunstwissenschaften
	Land- und Forstwirtschaft, Fischerei
	Maschinenbau
	Mathematik
	Medien- und Kommunikationswi

	Umweltingenieurwesen, Angewandte Geowissenschaften
	Veterinärmedizin
	Werkstofftechnik
	Wirtschaftswissenschaften
Stiftungen/Fonds/Sonstige Fördereinrichtungen
	Gesamt
	Agrarbiotechnologie, Lebensmittelbiotechnologie
	Andere Agrarwissenschaften
	Andere Geisteswissenschaften
	Andere Humanmedizin, Gesundheitswissenschaften
	Andere Naturwissenschaften
	Andere Sozialwissenschaften
	Andere Technische Wissenschaften
	Architektur
	Bauwesen
	Bildende Kunst
	Biologie
	Bühnengestaltung
	Chemie
	Chemische Verfahrenstechnik
	Computermusik
	Design
	Elektrotechnik, Elektronik, Informationstechnik
	Erziehungswissenschaften
	Film und Fernsehen
	Geowissenschaften
	Geschichte, Archäologie
	Gesundheitswissenschaften
	Humangeographie, Regionale Geographie, Raumplanung
	Industrielle Biotechnologie
	Informatik
	Interpretation - instrumental
	Interpretation - vokal
	Jazz/Improvisation
	Klinische Medizin
	Komposition
	Konservierung und Restaurierung
	Kunstwissenschaften
	Land- und Forstwirtschaft, Fischerei


	Umweltingenieurwesen, Angewandte Geowissenschaften
	Veterinärmedizin
	Werkstofftechnik
	Wirtschaftswissenschaften
Jubiläumsfonds der ÖNB
	Gesamt
	Agrarbiotechnologie, Lebensmittelbiotechnologie
	Andere Agrarwissenschaften
	Andere Geisteswissenschaften
	Andere Humanmedizin, Gesundheitswissenschaften
	Andere Naturwissenschaften
	Andere Sozialwissenschaften
	Andere Technische Wissenschaften
	Architektur
	Bauwesen
	Bildende Kunst
	Biologie
	Bühnengestaltung
	Chemie
	Chemische Verfahrenstechnik
	Computermusik
	Design
	Elektrotechnik, Elektronik, Informationstechnik
	Erziehungswissenschaften
	Film und Fernsehen
	Geowissenschaften
	Geschichte, Archäologie
	Gesundheitswissenschaften
	Humangeographie, Regionale Geographie, Raumplanung
	Industrielle Biotechnologie
	Informatik
	Interpretation - instrumental
	Interpretation - vokal
	Jazz/Improvisation
	Klinische Medizin
	Komposition
	Konservierung und Restaurierung
	Kunstwissenschaften
	Land- und Forstwirtschaft, Fischerei
	Maschinenbau
	Mathemat

/home/david/Storage/Computing/Python/datascience/jupyter/scraper/1.C.1_Fördergeber_WZW.csv

Output()

<IPython.core.display.Javascript object>

In [13]:
import os, time, datetime, locale

path = '.'
output = widgets.Output()
display(widgets.HTML('<h3>Vorhandene Daten</h3>'), output)
for ind_name in indicators.keys():
    files = [i for i in os.listdir(path) if ind_name in i and '.csv' in i]
    for file in files:
        modified = datetime.datetime.fromtimestamp(os.path.getmtime(file))
        prefix = str(modified.date()) + ": "
        display(FileLink(file, result_html_prefix=prefix))
        

HTML(value='<h3>Vorhandene Daten</h3>')

Output()

/home/david/Storage/Computing/Python/datascience/jupyter/scraper/1.C.1_Fördergeber_WZW.csv